In [ ]:

# ✅ LSTM 4개 Feature 버전 - 깔끔 주석 포함

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1️⃣ 데이터 로드
df = pd.read_csv('목적별_국적별_결측치.csv')

# 2️⃣ 결측치 처리
df = df.fillna(method='ffill')
df = df.sort_values(['country_code', 'purpose_code', 'year', 'month'])

# 3️⃣ Feature 4개만 사용
features = ['visitors_num', 'lag_1', 'rolling_mean_3', 'is_peak']

# 4️⃣ 스케일링
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

# 5️⃣ Sliding Window
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps, 0])  # visitors_num
    return np.array(X), np.array(y)

n_steps = 12
X, y = create_sequences(scaled, n_steps)

print(f'X Shape: {X.shape}')
print(f'y Shape: {y.shape}')

# 6️⃣ LSTM
n_features = X.shape[2]

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X, y, epochs=50, batch_size=32)

# 7️⃣ 예측
y_pred = model.predict(X)
y_pred_inv = scaler.inverse_transform(
    np.hstack([y_pred, np.zeros((len(y_pred), len(features)-1))])
)[:,0]

print('예측 샘플:', y_pred_inv[:5])

# 8️⃣ 저장
model.save('foreign_visitors_lstm_4features.h5')
print('✅ 4개 Feature LSTM 모델 저장 완료')
